1. Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:

*   взять в тренировочный набор турниры с dateStart из 2019 года;
*   взять в тренировочный набор турниры с dateStart из 2019 года;


In [1]:
import zipfile
import pickle

! wget https://www.dropbox.com/s/s4qj0fpsn378m2i/chgk.zip -nc

with zipfile.ZipFile('chgk.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

tournaments = pickle.load(open('tournaments.pkl', 'rb'))
results = pickle.load(open('results.pkl', 'rb'))

File ‘chgk.zip’ already there; not retrieving.



In [ ]:
for id, data in tournaments.items():
  print(data)

In [4]:
def get_data_ids(year):
  train = []
  for id, data in tournaments.items():
    if data["dateStart"][:4] == year:
      train.append(data)
  train_2019_with_results = []
  for idx, data in enumerate(train):
    if data['id'] in results and results[data['id']] != []:
      train_2019_with_results.append(data)
  train_2019_with_results_mask = []
  for idx, data in enumerate(train_2019_with_results):
    if 'mask' in results[data['id']][0]:
      train_2019_with_results_mask.append(data)
  ids = []
  for i in train_2019_with_results_mask:
    ids.append(i["id"])
  return ids

In [5]:
train_ids = get_data_ids("2019")
test_ids = get_data_ids("2020")

In [6]:
print(len(train_ids))
print(len(test_ids))

679
176


In [7]:
def get_results(results, ids):
  data = {}
  for id in ids:
    data[id] = results[id]
  return data

res = get_results(results, test_ids + train_ids)

In [ ]:
res[4957]

2. Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. 
Замечания и подсказки:


*   повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
*   в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать;
*   скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
*   для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.

Решение:
Данные представляют с собой id игрока и сложность вопоосы. Id игрока хотелось закодировть в виде one-hot вектора, но к сожалению не хватило ОЗУ. Дальше планировалось обучить регрессию (линейную или логистичекую)

In [ ]:
count = 0
for i in res.items():
  if count == 1:
    break
  for elem in i[1]:
    print("team: ", elem["team"])
    print("mask: ", elem["mask"])
    print("current: ", elem["current"])
    print("questionsTotal: ", elem["questionsTotal"])
    print("synchRequest: ", elem["synchRequest"])
    print("position: ", elem["position"])
    print("controversials: ", elem["controversials"])
    print("flags: ", elem["flags"])
    print("teamMembers: ", elem["teamMembers"])
  count += 1

In [10]:
len(res[4957])

92

In [11]:
len(res)

855

In [12]:
import numpy as np

In [13]:
all_ids = train_ids + test_ids

In [16]:
import pandas as pd
import numpy as np


def get_dataset(ids):
    dataset = []
    for id in ids:
        teams = []
        masks = []        
        for team in res[id]:
            masks.append(team['mask'])
            team_members = []
            for member in team['teamMembers']:
                team_members.append(member['player']['id'])
            teams.append(team_members)

        vec_of_masks = np.zeros(len(masks[0]))
        for i, mask in enumerate(masks):
            for question_num, result in enumerate(mask):
                vec_of_masks[question_num] += int(str(result))
         
        team_count = len(results[id])
        vec_of_masks = vec_of_masks / team_count
        
        for i in vec_of_masks:
            i = i / team_count

        for question_id, vec in enumerate(vec_of_masks):
            for team_id, team in enumerate(teams):
                result = masks[team_id][question_id]
                for player in team:                    
                    dataset.append([player, vec, int(str(result))])
        return dataset

In [17]:
dataset = get_dataset(all_ids)

In [18]:
dataset[:5]

[[6212, 0.8831168831168831, 1],
 [18332, 0.8831168831168831, 1],
 [18036, 0.8831168831168831, 1],
 [22799, 0.8831168831168831, 1],
 [15456, 0.8831168831168831, 1]]

In [19]:
def vectorize(dataset, player_id_columns):
    X = []
    for row in dataset:
        one_hot_player = [0] * len(player_id_columns)
        one_hot_player[player_id_columns.index(row[0])] = 1.0
        one_hot_player.append(row[1])
        X.append(one_hot_player)
    return X

In [20]:
def get_all_players_id(all_ids):
    ids = []
    for id in all_ids:
        for team in res[id]:
            for player in team['teamMembers']:
                ids.append(player['player']['id'])
    return sorted(list(set(ids)))

In [21]:
players_ids = get_all_players_id(all_ids)

In [22]:
train_data = np.unique(get_dataset(train_ids), axis=0)
np.array(train_data).shape

(38850, 3)

In [ ]:
# не хватает ОЗУ
X_train = vectorize(train_data, players_ids)

3. Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:

*   предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
*   в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.





In [23]:
import pickle
results = pickle.load(open('results.pkl', 'rb'))
results = get_results(results, test_ids)

In [24]:
from collections import Counter
from scipy.stats import spearmanr, kendalltau
import math

5. А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в базе вопросов ЧГК (это чисто техническое дело, тут никакого ML нету).


In [29]:
results = pickle.load(open('results.pkl', 'rb'))
results = get_results(results, train_ids)

In [38]:
def get_masks(ids):
  good_ids = []
  for id in ids:
    all_questions = 0  
    for key, val in tournaments[id]["questionQty"].items():          
      all_questions += val
    masks = []     
    for team in results[id]:
      masks.append(team['mask'])
    good_mask = True
    for mask in masks:
      if mask == None:        
        good_mask = False
      elif len(mask) != all_questions:
        good_mask = False
    if good_mask:
      ids.append(id)
  return ids


def get_dataset(ids):
    dataset = {}
    for id in ids:
        all_questions = 0        
        for k,v in tournaments[id]['questionQty'].items():          
          all_questions += v 
        masks = []     
        for team in results[tournament_id]:
            if team['mask']:
              masks.append(team['mask'])
            else:
              masks.append("0" * all_questions)
        vec_of_masks = np.zeros(questions_total)
        for id, mask in enumerate(masks):
          for question_num, result in enumerate(mask):
            if result == '0' or result == '1':                
              vec_of_masks[question_num] += int(str(result))

        count = 0
        for elem in vec_of_masks:
            count += elem             
        dataset[id] = count
        
    return dataset

In [39]:
ids = get_masks(train_ids)
tournamens_comp = get_dataset(ids)
print(tournamens_comp)

KeyboardInterrupt: ignored